In [1]:
from active_contour_model import *

In [ ]:
img = cv2.imread("D:\Study\object-collision\Active_Contour_Snake\Item\object.png")
img = cv2.resize(img, None, fx=0.9, fy=0.9)
# cv2.imshow("image test", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb)
plt.title('Image_test')
plt.axis('off')

In [3]:
gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
contours, hierarchy = cv2.findContours(binary, mode= cv2.RETR_LIST, method= cv2.CHAIN_APPROX_TC89_KCOS)

In [ ]:
print(contours)

In [5]:
image_copy = img.copy()
image_copy = cv2.drawContours(image_copy, contours, -1, (0, 255, 0), thickness= 5, lineType = cv2.LINE_AA)

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(1,3,1); plt.imshow(gray, cmap= "gray");plt.title('Grayscale Image');plt.axis('off')
plt.subplot(1,3,2); plt.imshow(binary, cmap= "gray");plt.title('Binary Image');plt.axis('off')
plt.subplot(1,3,3); plt.imshow(image_copy, cmap= "gray");plt.title('Drawn Contours');plt.axis('off')

In [7]:
video_path = r"D:\Study\object-collision\Active_Contour_Snake\Item\Object collision - Made with Clipchamp.mp4"

In [8]:
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)

30.0


In [9]:
cv2.imshow("Binary Image", binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
output_path = r"Output_video.mp4"

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

t = TicToc()
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    t.tic()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_TC89_KCOS)

    cv2.drawContours(frame, contours, -1, (0, 255, 0), 5)

    out.write(frame)

    #cv2.imshow("Contours frame", frame)

    if cv2.waitKey(0) != -1:
        break
t.toc()
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed {frame_count} frames.")
print(f"Output video saved to {output_path}")

Elapsed time is 0.005955 seconds.
Processed 129 frames.
Output video saved to Output_video.mp4


***LỖI VIDEO***

In [12]:
from scipy.spatial import cKDTree

def calculate_contour_distance(contour1, contour2):
    """Tính khoảng cách nhỏ nhất giữa hai contours bằng KD-Tree."""
    points1 = np.squeeze(contour1)  # Loại bỏ chiều không cần thiết
    points2 = np.squeeze(contour2)
    
    if len(points1.shape) == 1:
        points1 = points1[np.newaxis, :]
    if len(points2.shape) == 1:
        points2 = points2[np.newaxis, :]
    
    tree = cKDTree(points2)
    distances, _ = tree.query(points1, k=1)
    return distances.min()

video_path = r"D:\Study\OpenCV_Tomi\object collision\Item\Object collision - Made with Clipchamp.mp4"
cap = cv2.VideoCapture(video_path)
output_path = r"Output_video.mp4"

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)

# Khởi tạo video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec cho video
out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

frame_count = 0
collision_frame = -1  # Khung hình xảy ra va chạm
collision_threshold = 355  # Ngưỡng va chạm theo pixel

t = TicToc()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    t.tic()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_TC89_KCOS)

    if len(contours) >= 2:
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]
        contour1, contour2 = contours

    min_distance = calculate_contour_distance(contour1, contour2)
    print(f"Frame {frame_count}: Min Distance = {min_distance}")

    # Kiểm tra va chạm
    if min_distance <= collision_threshold:
        collision_frame = frame_count
        print(f"Collision detected at frame: {collision_frame}")
        break

    # Vẽ contours trên frame
    cv2.drawContours(frame, [contour1], -1, (0, 255, 0), 5)
    cv2.drawContours(frame, [contour2], -1, (0, 0, 255), 5)
else:
    print(f"Frame {frame_count}: Not enough contours detected")
    out.write(frame)

cap.release()
out.release()

# In kết quả
if collision_frame != -1:
    collision_time = collision_frame / fps
    print(f"Collision occurred at frame {collision_frame} ({collision_time:.2f} seconds).")
else:
    print("No collision detected.")

Frame 1: Min Distance = 571.1357106677887
Frame 2: Min Distance = 571.1357106677887
Frame 3: Min Distance = 571.1357106677887
Frame 4: Min Distance = 571.1357106677887
Frame 5: Min Distance = 571.1357106677887
Frame 6: Min Distance = 571.1357106677887
Frame 7: Min Distance = 571.1357106677887
Frame 8: Min Distance = 571.1357106677887
Frame 9: Min Distance = 571.1357106677887
Frame 10: Min Distance = 571.1357106677887
Frame 11: Min Distance = 571.1357106677887
Frame 12: Min Distance = 571.1357106677887
Frame 13: Min Distance = 571.1357106677887
Frame 14: Min Distance = 571.1357106677887
Frame 15: Min Distance = 520.0038461396223
Frame 16: Min Distance = 520.0038461396223
Frame 17: Min Distance = 506.0088932024812
Frame 18: Min Distance = 492.0091462564492
Frame 19: Min Distance = 478.0167361086848
Frame 20: Min Distance = 464.02693887316497
Frame 21: Min Distance = 450.0277769204919
Frame 22: Min Distance = 450.0277769204919
Frame 23: Min Distance = 435.0735569992734
Frame 24: Min Dista

***LỖI DRAW CONTOURS --> KHÔNG XÁC ĐỊNH ĐƯỢC DETECT --> VẪN TÍNH ĐƯỢC ELAPSED TIME CHƯA PHẢI TIME CHẠM NHAU***

In [13]:
output_path = r"Output_video.mp4"

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

t = TicToc()
frame_count = 0
collision_frame = -1
collision_threshold = 10  # Ngưỡng va chạm theo pixel

t.tic()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    contours = preprocess_frame(frame)
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # ret, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # contours, hierarchy = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_TC89_KCOS)

    if len(contours) >= 2:
        # Lấy hai contours lớn nhất
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]
        centroid1 = calculate_centroid(contours[0])
        centroid2 = calculate_centroid(contours[1])

        if detect_collision(centroid1, centroid2, radius1=0, radius2=0):
            collision_frame = frame_count
            print(f"Collision detected at frame {collision_frame} ({collision_frame / fps:.2f} seconds).")
            break

        cv2.drawContours(frame, [contours[0]], -1, (0, 255, 0), 2)
        cv2.drawContours(frame, [contours[1]], -1, (0, 0, 255), 2)
        if centroid1:
            cv2.circle(frame, centroid1, 5, (255, 0, 0), -1)
        if centroid2:
            cv2.circle(frame, centroid2, 5, (0, 255, 255), -1)

    out.write(frame)

cap.release()
out.release()

t.toc()
cv2.destroyAllWindows()

# In thông tin va chạm
if collision_frame != -1:
    print(f"Collision detected at frame {collision_frame} ({collision_frame / fps:.2f} seconds).")
else:
    print("No collision detected.")

Elapsed time is 0.000200 seconds.
No collision detected.
